In [ ]:
import hashlib
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import smtplib
from email.mime.text import MIMEText


"""
TODO:

1. Add a way to get new text when a website is added 
1a. A way to find the difference between new text or old text. 
        - use the hexdigest
2. A way to store the text 
        - use another dictionary
3. A way to append the text and the links to the email. 
4. A way to download files from a file and attach it to the email. 
5. A way to process the new links (go to link and grab the text/ grab potential documents, or if it is a documetn
    it will send to the recipient. 

"""

class Monitor():

    #recipient_email = 'dan@turbinehub.com'
    #password = 'fugquz-sohne0-Nigmyw'
    password = 'pxkqlllkkwgwfszm'
    #sender_email = 'dylan@turbinehub.com'
    sender_email = 'dan@turbinehub.com'
    recipient_email = 'dylan@turbinehub.com'
    #will recieve a url as a key, and a hash as a value
    previous_hash = dict()
    #will recieve a url as a key and a list of links as a value
    previous_links = dict()
    new_links = list()
    base_link = ''
    url_list = list()
    text_list = dict()

    def __init__(self,url_list, base_link):
        self.url_list = url_list
        self.base_link = base_link


    def send_email(self, url=None, diff_text=None):
        body = 'There is a change on ' + self.base_link + '\n'
        if url:
            for link in url:
                if 'www' in link or 'http' in link:
                    body += link + '\n'
                else:
                    body += self.base_link + link + ' \n'
            body += 'If there is no URL, then there is a change to the text'
        else:
            body += 'There is no new links to this change. Please check the website'

        if diff_text:
            body += diff_text

        message = MIMEText(body, 'plain')
        message['From'] = self.sender_email
        message['To'] = self.recipient_email
        message['Subject'] = 'URGENT!!! NEW CHANGE DETECTED'
        #this will create a new session
        session = smtplib.SMTP('smtp.gmail.com', 587)
        session.starttls()
        session.login(self.sender_email, self.password)
        text = message.as_string()
        session.sendmail(self.sender_email, self.recipient_email , text)

    def detect_change(self):
        while True:
            print(self.url_list)
            for url in self.url_list:
                    request_url = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                    response = urlopen(request_url).read()
                    current_hash = hashlib.sha224(response).hexdigest()
                    soup = BeautifulSoup(response, 'lxml')
                    text_soup = BeautifulSoup(response.text, 'html.parser')
                    text_soup = text_soup.get_text(' ', strip=True)
                    all_links = self.get_links(soup)
                    if url not in self.previous_hash:
                        print('adding previous hash')
                        self.previous_links[url] = all_links
                        self.previous_hash[url] = current_hash
                        self.text_list[url] = text_soup
                    elif current_hash != self.previous_hash.get(url):
                        print('Change detected')
                        diff = self.find_differences(all_links, url)
                        diff_text = set(self.text_list[url]).difference(text_soup)
                        self.send_email(diff, url, diff_text)
                        self.previous_links[url] = all_links
                        self.previous_hash[url] = current_hash
                    else:
                        print('No changes found for ' + url)


    def find_differences(self, new_links, url):
        diff = new_links.difference(self.previous_links[url])
        return diff

    def get_links(self, soup):
        links = set()
        for link in soup.findAll('a'):
            links.add(link.get('href'))
        print(links)
        return links